# 生成器
generator一边循环一边计算洗衣歌元素的机制/算法

In [3]:
#直接使用生成器
L = [x*x for x in range(5)]  #放在中括号中是列表生成器
G = (x*x for x in range(5))  #放在小括号中是生成器

print(type(L))
print(type(G))

<class 'list'>
<class 'generator'>


In [4]:
#函数案例
def odd():
    print("第一步")
    print("第二步")
    print("第三步")
    print("第四步")
    return None
odd()

第一步
第二步
第三步
第四步


In [44]:
#生成器案例
#在函数odd中,yield负责返回

def odd():
    print("第一步")
    yield 1
    print("第二步")
    yield 1
    print("第三步")
    yield 1
    print("第四步")
    yield 1
a = odd()
ha = next(a)
ha
hb = next(a)
hb
hc = next(a)
hc
hd = next(a)

第一步
第二步
第三步
第四步


In [52]:
#for 循环调用生成器
def fib(max):
    n , a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a+b 
        n += 1
    return None

x = fib(9)

for i in range(9+1):
    rst = next(x)  #next 调用
    print(rst)

1
1
2
3
5
8
13
21
34


StopIteration: 

In [54]:
num = int(input("输入一个数:"))
a  = fib(num)
for i in range(num):
    s = next(a)
    print(s)

输入一个数:9
1
1
2
3
5
8
13
21
34


# 协程
- 协程的历史
    - 3.4引入协程概念，用yield实现
    - 3.5引入协程语法
    - 实现协程比较好的包有asyncio，tornado，gevent
- 定义：“协程是为非抢占式多任务产生子程序的计算机程序组件，协程允许不同入口点在不同位置暂停或开始执行程序”。从技术的角度来说，“协程就是你   可以暂停执行的函数“。如果你把它理解成”就像生成器一样“，那么你就想对了。
- 协程的实现
    - yield 返回
    - send 调用
- 协程的四个状态
        -inspect.getgeneratorstate（...）函数确定，该函数会返回下述字符串中的一个：
    -GEN_CREATED：等待开始执行
    -GEN_RUNNING：解释器正在执行
    -GEN_SUSPENED：在产量表达式处暂停
    -GEN_CLOSED：执行结束
    -接下来的预激（黄金）

-完整执行过程V2

- 协程终止
    -协程中未处理的异常会向上冒泡，传给next函数或send方法的调用方（即触发协程的对象）。
    
    -终止协程的一种方式：发送某个哨符值，让协程退出。内置的无和Ellipsis等常量经常用作哨符值==。

- yield from 
    - 调用协程为了得到返回值,协程必须正常终止
    -生成器正常终止会发出 StopIteration异常 ,异常对象的vlaue属性保存返回值
    -yield from 从内部捕获StopIteration异常
    -案例3
    -委派生成器
        -包含yield from 表达式的生成器
        -委派生成器在yield from 表达式出暂停 ,调用方可以直接发给自生成器
        -子生成器在把产出的值发给调用方
        -自生成器在最后,解释器会抛出StopIteration,并且把返回值附加到异常对象上

In [75]:
#协程代码案例1
def simple_coroutine():
    print('->start')
    x = yield
    print('->recived',x)
#主线程
sc = simple_coroutine()
print(111)
#可以使用sc.send(None),效果一样
next(sc) #预激
sc.send("kkkk")

111
->start
->recived kkkk


StopIteration: 

In [79]:
#协程案例2
def simple_coroutine(a):
    print('-> start')
    b = yield a
    print('->recived', a,b)
    c = yield a+b
    print('->recived', a, b, c)
    
q = simple_coroutine(5)
a = next(q)
print(a)
b = q.send(9)
print(b)
c = q.send(2)
print(c)

-> start
5
->recived 5 9
14
->recived 5 9 2


StopIteration: 

In [85]:
#案例3
def gen():
    for c in "AB":
        yield c
print(list(gen()))
# list 直接做生成参数
def gen_new():
    yield from "AB"
print(list(gen_new()))

['A', 'B']
['A', 'B']


In [87]:
#案例4 委派生产器

In [1]:
# 案例v04， 委派生成器
from collections import namedtuple

'''
#计算平均身高和体重
解释：
1. 外层 for 循环每次迭代会新建一个 grouper 实例，赋值给 coroutine 变量； grouper 是委派生成器。
2. 调用 next(coroutine)，预激委派生成器 grouper，此时进入 while True 循环，调用子生成器 averager 后，在 yield from 
   表达式处暂停。
3. 内层 for 循环调用 coroutine.send(value)，直接把值传给子生成器 averager。同时，当前的 grouper 实例（coroutine） 
   在 yield from 表达式处暂停。
4. 内层循环结束后， grouper 实例依旧在 yield from 表达式处暂停，因此， grouper函数定义体中为 results[key] 赋值的语句还没有执行。
5. coroutine.send(None) 终止 averager 子生成器，子生成器抛出 StopIteration 异常并将返回的数据包含在异常对象的value中，
   yield from 可以直接抓取 StopItration 异常并将异常对象的 value 赋值给 results[key]
'''
ResClass = namedtuple('Res', 'count average')


# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None

    while True:
        term = yield
        # None是哨兵值
        if term is None:
            break
        total += term
        count += 1
        average = total / count

    return ResClass(count, average)


# 委派生成器
def grouper(storages, key):
    while True:
        # 获取averager()返回的值
        storages[key] = yield from averager()


# 客户端代码
def client():
    process_data = {
        'boys_2': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        'boys_1': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
    }

    storages = {}
    for k, v in process_data.items():
        # 获得协程
        coroutine = grouper(storages, k)

        # 预激协程
        next(coroutine)

        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)

        # 终止协程
        coroutine.send(None)
    print(storages)

# run
client()

{'boys_2': Res(count=9, average=40.422222222222224), 'boys_1': Res(count=9, average=1.3888888888888888)}


# 还剩下的内容
- xml json
- re，xpath
- 网络编程：socket，ftp，mail
- http协议==>http web sever 小项目
- diango
---------
-


# asyncio
- python3.4开始引入标准库，内置对异步io的支持
- asyncio 本身是一个消息循环
- 步骤
    - 创建一个消息循环
    - 把协程导入
    - 关闭

In [1]:
import threading
# 引入异步io包
import asyncio
@asyncio.coroutine
# 使用协程
def hello():
    print('hello world! (%s)' % threading.currentThread())
    print('start..... (%s)' % threading.currentThread())
    yield from asyncio.sleep(10)
    print('Done..... (%s)' % threading.currentThread())
    print('hello again (%s)' % threading.currentThread())
#启用消息循环
loop = asyncio .get_event_loop()
#定义任务
tasks = [hello(),hello()]
# asyncio 使用wait等待task执行完毕
loop.run_until_complete(asyncio.wait(tasks))
# 关闭消息循环
loop.close()

hello world! (<_MainThread(MainThread, started 140621253187328)>)
start..... (<_MainThread(MainThread, started 140621253187328)>)
hello world! (<_MainThread(MainThread, started 140621253187328)>)
start..... (<_MainThread(MainThread, started 140621253187328)>)
Done..... (<_MainThread(MainThread, started 140621253187328)>)
hello again (<_MainThread(MainThread, started 140621253187328)>)
Done..... (<_MainThread(MainThread, started 140621253187328)>)
hello again (<_MainThread(MainThread, started 140621253187328)>)


# async and await
- 为了更好的表示异步
- python3.5引入
- 让程序更加简洁
- 使用上，可以简单的进行替换
    - 用async 替换@asyncio.coroutine
    - await 替换 yield from
   

In [2]:
import threading
# 引入异步io包
import asyncio
#@asyncio.coroutine
# 使用协程
async def hello():
    print('hello world! (%s)' % threading.currentThread())
    print('start..... (%s)' % threading.currentThread())
    await asyncio.sleep(10)
    print('Done..... (%s)' % threading.currentThread())
    print('hello again (%s)' % threading.currentThread())
#启用消息循环
loop = asyncio .get_event_loop()
#定义任务
tasks = [hello()]
# asyncio 使用wait等待task执行完毕
loop.run_until_complete(asyncio.wait(tasks))
# 关闭消息循环
loop.close()

RuntimeError: Event loop is closed

# aiohttp
- asyncio 实现单线程的并发io，在客户端用处不大
- 在服务器端可以asyncio+coroutine配合，因为http是io操作
- asyncio实现了tcp，udp，ssl等协议
- aiohttp是给予asyncio实现http框架
- pip install aiohttp安装


In [2]:
#案例
import asyncio

from aiohttp import web

async def index(request):
    await asyncio.sleep(0.5)
    return web.Response(body=b'<h1>Index</h1>')

async def hello(request):
    await asyncio.sleep(0.5)
    text = '<h1>hello, %s!</h1>' % request.match_info['name']
    return web.Response(body=text.encode('utf-8'))

async def init(loop):
    app = web.Application(loop=loop)
    app.router.add_route('GET', '/', index)
    app.router.add_route('GET', '/hello/{name}', hello)
    srv = await loop.create_server(app.make_handler(), '127.0.0.1', 8000)
    print('Server started at http://127.0.0.1:8000...')
    return srv

loop = asyncio.get_event_loop()
loop.run_until_complete(init(loop))
loop.run_forever()


ModuleNotFoundError: No module named 'aiohttp'

In [6]:
from concurrent.futures import ThreadPoolExecutor
import time


def return_future(msg):
    time.sleep(3)
    return msg


# 创建一个线程池
pool = ThreadPoolExecutor(max_workers=2)

# 往线程池加入2个task
f1 = pool.submit(return_future, 'hello')
f2 = pool.submit(return_future, 'world')

print(f1.done())
time.sleep(3)
print(f2.done())

print(f1.result())
print(f2.result())

False
False
hello
world


In [1]:
from concurrent.futures import ThreadPoolExecutor as Pool
#import requests
import urllib
from urllib import request

URLS = ['http://www.baidu.com', 'http://qq.com', 'http://sina.com']


def task(url, timeout=20):
    #return requests.get(url, timeout=timeout)
    return request.urlopen(url, timeout=timeout)


pool = Pool(max_workers=3)
results = pool.map(task, URLS)

import time
time.sleep(20)
for ret in results:
    print('%s, %s' % (ret.url, len(ret.read())))
    

http://www.baidu.com, 118941
https://www.qq.com/?fromdefault, 44094


URLError: <urlopen error timed out>

# current